In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


We will use the CIFAR-10 dataset for this example. The CIFAR-10 dataset consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. We will preprocess the data by scaling the pixel values to the range [0, 1] and converting the labels to one-hot encoding:

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess the data
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
#categorical encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


Next, we will define the pre-trained models that we want to use for transfer learning. We will use the include_top=False argument to remove the fully connected layers from the models, since we will be adding our own output layer. We will also specify the input_shape argument to match the input shape of the CIFAR-10 dataset:

In [3]:
# Define the pre-trained models
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
vgg19 = VGG19(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))


94765736/94765736 [==============================] - 103s 1us/step


We will now define the output layers for each model. Since we are using the CIFAR-10 dataset, we will add a global average pooling layer to reduce the spatial dimensions of the output feature map, followed by a dense output layer with 10 units and softmax activation for classification:

In [4]:
# Define the output layers
vgg16_output = layers.GlobalAveragePooling2D()(vgg16.output)
vgg16_output = layers.Dense(10, activation="softmax")(vgg16_output)

vgg19_output = layers.GlobalAveragePooling2D()(vgg19.output)
vgg19_output = layers.Dense(10, activation="softmax")(vgg19_output)

resnet_output = layers.GlobalAveragePooling2D()(resnet.output)
resnet_output = layers.Dense(10, activation="softmax")(resnet_output)


In [5]:
# Define the complete models
vgg16_model = keras.Model(inputs=vgg16.input, outputs=vgg16_output)
vgg19_model = keras.Model(inputs=vgg19.input, outputs=vgg19_output)
resnet_model = keras.Model(inputs=resnet.input, outputs=resnet_output)


In [6]:
# Compile the models
vgg16_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
vgg19_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
resnet_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Evaluate the models
vgg16_loss, vgg16_accuracy = vgg16_model.evaluate(x_test, y_test, verbose=0)
vgg19_loss, vgg19_accuracy = vgg19_model.evaluate(x_test, y_test, verbose=0)
resnet_loss, resnet_accuracy = resnet_model.evaluate(x_test, y_test, verbose=0)

# Print the results
print("VGG16 Test Accuracy:", vgg16_accuracy)
print("VGG19 Test Accuracy:", vgg19_accuracy)
print("ResNet50 Test Accuracy:", resnet_accuracy)


VGG16 Test Accuracy: 0.08919999748468399
VGG19 Test Accuracy: 0.13680000603199005
ResNet50 Test Accuracy: 0.09189999848604202
